# test from youtube tutorial

In [ ]:
from huggingface_hub import login
login(token="my-login-token")

In [9]:
from transformers import pipeline
from transformers.image_utils import load_image

In [10]:
image = load_image("/Users/varyabazilova/Desktop/glacial_lakes/super_lakes/dinov3_tryout/test_data/2021-09-04_fcc.tif")
feature_extractor = pipeline(
    model = "facebook/dinov3-vitb16-pretrain-lvd1689m",
    task = "image-feature-extraction",
)

features = feature_extractor(image)


Device set to use cpu


In [15]:
# features

In [17]:
# Check the structure first
print(f"Features type: {type(features)}")
print(f"Features length: {len(features)}")
print(f"First element type: {type(features[0])}")

# Get the actual feature tensor
import torch
import numpy as np

# Extract the tensor from the nested structure
feature_tensor = torch.tensor(features[0])
print(f"Feature tensor shape: {feature_tensor.shape}")

# Convert to numpy for analysis
features_array = feature_tensor.numpy()
print(f"Features array shape: {features_array.shape}")

Features type: <class 'list'>
Features length: 1
First element type: <class 'list'>
Feature tensor shape: torch.Size([201, 768])
Features array shape: (201, 768)


In [ ]:
feature_extractor = pipeline(
  model="facebook/dinov3-vit7b16-pretrain-sat493m",  # Satellite-trained version
  task="image-feature-extraction",
)

features = feature_extractor(image)

Fetching 6 files:   0%|                                                                                                                                                             | 0/6 [05:07<?, ?it/s]


# create patches 

In [ ]:
import numpy as np
from PIL import Image
import torch

def create_patches(image_path, patch_size=224):
  """Cut image into small squares for analysis"""

  # Load your satellite image
  image = Image.open(image_path)
  img_array = np.array(image)

  patches = []
  positions = []  # Remember where each patch came from

  height, width = img_array.shape[:2]

  # Cut image into patches
  for y in range(0, height-patch_size, patch_size):
      for x in range(0, width-patch_size, patch_size):
          # Extract small square
          patch = img_array[y:y+patch_size, x:x+patch_size]

          patches.append(patch)
          positions.append((y, x))  # Save position

  return patches, positions, (height, width)

# Use it on your image
patches, positions, original_size = create_patches("/path/to/your/image.tif")
print(f"Created {len(patches)} patches from your image")

# Now you can analyze each patch with DINOv3:

# Analyze each patch
patch_features = []
for patch in patches:
  patch_pil = Image.fromarray(patch)
  features = feature_extractor(patch_pil)
  patch_features.append(features[0])
